In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore, db
import json

In [2]:
#cred = credentials.Certificate('/Users/evanrussek/forage_jsp/analysis/foragetask-firebase-private-key.json')
cred = credentials.Certificate('C:\\Users\\erussek\\forage_jsp\\analysis\\foragetask-firebase-private-key.json')
default_app = firebase_admin.initialize_app(cred)
client = firestore.client()

In [9]:
dfs = []
bonus_df = pd.DataFrame()
for subj in client.collection('foragetask').document('run5').collection('subjects').stream():
    Qdfs = []
    subj_dfs = []
    taskdata_collection = client.collection('foragetask/run5/subjects/{0}/taskdata'.format(subj.id)).stream()
    has_end = False # did this subject finish the task?\n",
    has_start = False
    subjectID = subj.id
    for tc in taskdata_collection:
        if tc.id == "start":
            has_start = True
            start_dict = tc.to_dict()
            subjectID = start_dict['subjectID']
        elif tc.id == "end":
            print(subj.id) # ok. it's loopint through this too many times...............
            has_end = True
            end_dict = tc.to_dict()
            end_time = end_dict['end_time']
            bonus_points = end_dict['bonus_points'][0]
        elif tc.id == "Q_data":
            has_start = True
            Q_dict = tc.to_dict()
            Q_data_json = json.loads(Q_dict['trial_data'])
            Q_df = pd.DataFrame(Q_data_json)
            Q_df['subjectID'] = subj.id
            Qdfs.append(Q_df)
        else:
            trial_dict = tc.to_dict()
            trial_name = tc.id
            trial_data_json = json.loads(trial_dict['trial_data'])
            trial_df = pd.DataFrame(trial_data_json)
            trial_df['subjectID'] = subjectID
            if has_start:
                trial_df['subjectID'] = subjectID
                #trial_df['date'] = date
            else:
                trial_df['subjectID'] = None
                #trial_df['date'] = None
            if has_end:
                trial_df['bonus_points'] = bonus_points
            else:
                trial_df['bonus_points'] = None
            subj_dfs.append(trial_df)
    if (has_end & (len(str(subjectID)) > 20)):
        bonus_dict = {'subjectID': subjectID, 'bonus_points': bonus_points}
        bonus_df = bonus_df.append(bonus_dict, ignore_index = True)
        dfs = dfs + subj_dfs
combined_df = pd.concat(dfs, sort = True)
combined_df.to_csv('data/run5_data.csv')

0tz8Cw7eVkTKuO2UHapNknfteQJ2
3SI32LAtgtSPUJoSXCZpvJdzwf83
6TPp2mSQ0hRGyHnCzrPXemVJzTW2
AQ6D3QZ1MlUTy2W5kAe4G1qrzGG3
DLpoDbT2TvX5dadfHpl02NQKQrY2
EYEqFGpurTOh6WC77qXeOoTKR9S2
GBIsqwfWnJdovMQohrgDH5h1En42
GSZ78YOlHoNsKOhOebWzTlFn1qt1
H0jPOy8vyyVXlymHjYJkmB4BCtO2
HONcxHvqn2hQGdSlCE8rccN5vW73
IqlID2KvgGR4bB4BU2TdYby5PC23
JNTDZTyQ6TTJgF5ah2fbT8fcLxj2
K9B7Sqii29bcFVBDBI6xFFh2ejR2
LXSGIiGuJtNx686E7Tbo3itaOoE3
MVr7BUv2UsM7IEHNPQtCkWLeJIB3
MWDCCvAOhkaeRDeEh6Zy1QWp22t2
PIgj5gk27pSccQmdPUAGPSFKBtg2
PjboZsy6D1dVFR4AgyTSCBkFjQz2
QUgzjuKjOXXI5Oo3fn9gvm1odW53
RQVt2D48a2UTiBbK7d1r71dAn633
SSqxANlQQHe4i1b8f6obWRiyBcz1
U7S37u6xYGSh2mJXqcaSbTVqvst2
Vqgfw0lFe5PTaYVKnZJAn8rawdI2
WEiZLxNmo8WhAnvO0IlgBwtC1Nv2
Xmf0up8ofRUbwnSLToRvsNBV21E2
ZPps9gBHm3OklDx3X9O4sPnPkUz1
bQYcFhQ1yshHOCS9os13nHKKVEn2
bXa63XJu9KNzYbuWTTeayCtVI4Z2
c0Cqg9GapnfN3tqOu81khLy5R2L2
eT4Ilf2EBcYJtedOlfpnYzWbApp1
elKtjWyJMKNufwk3zkDOBuhnBrh2
fCeOIQY1IAZKBp8PpO1KeBRciE43
gk98AQeDl3hvVTHd2z3FsujTplD2
iXFnIp9Y00S7o5r67SgwDGhO4Xx2
l88INCGpn1Pqnd

In [10]:
def compute_bonus(row):
    pct = np.min([np.max([row['bonus_points'] / (30000 - 0),0]),1])
    return np.around(2.50*pct, decimals = 2)
bonus_df['bonus'] = bonus_df.apply(compute_bonus, axis = 1)

In [11]:
bonus_df
bonus_df[['subjectID', 'bonus']].to_csv('bonus_payment_files\\run5_bonus_payments.csv', sep = ',', header = False, index = False)

In [7]:
bonus_dict

{'subjectID': '5c195dd90084c50001ed0ce5', 'bonus_points': 9585}

In [12]:
bonus_df

,bonus_points,subjectID,bonus
0,20879.0,5accded9a3ba7a0001b504e5,1.74
1,13714.0,5bc08c345e8a950001892156,1.14
2,10894.0,5d6183f7d989a40016236acc,0.91
3,21533.0,5dc5da21d999de45a504651b,1.79
4,25147.0,5d4b1ad811499700017452cb,2.10
5,17593.0,5db2d211841590000c0306da,1.47
6,6065.0,5cbc7935dc8a8900017390a2,0.51
7,20925.0,5af33167dd84460001c1dfcf,1.74
8,17534.0,5a4a93ab35c9d50001fdd082,1.46
9,11100.0,5d7fdc371c09e60001dd48f5,0.92


In [10]:
np.max(bonus_df['bonus_points'])

32125.0

In [17]:
30000/2.5

12000.0

In [ ]:
10000 points is a pound